<h1> Réalisation d’un mini solveur de CSP binaires</h1>
Master MPRO Pimprenelle Parmentier - Jonathan Zribi
Janvier 2020

## Données

In [34]:
# -*- coding: utf-8 -*-
import time
import timeit
import random

#exemple
variables=dict()
variables={}
variables["x1"]=[1,2]
variables["x2"]=[1,2]
contraintes=[[("x1","x2"),[(1,2),(2,2)]]]
instance=dict()
instance={}
#instance["x1"]=2
#instance["x2"]=1
#0: faux, 1: vérifié 2: incomplet

## Backtrack

In [91]:
"""
variables["x1"]=1
variables["x2"]=2
contraintes=[[("x1","x2"),[(1,2),(2,2)]]]
"""
#checkconstraint sur une instance partielle
#Instance partielle: couple de valeurs pour x1, x2.
#{'x1':1,'x2':2}
#Pour tout checker, for loop sur les instances partielles, contraintes.
# ici constraint est de la forme [('x1', 'x2'), [(1, 2), (2, 2)]
# par ex contraintes[0]
def checkConstraint(constraint,instance):
    var1=constraint[0][0]
    var2=constraint[0][1]
    #print('var1 check: ',var1)
    #print('var2 check: ',var2)
    #print('instance keys ', instance.keys())
    if var1 in instance.keys() and var2 in instance.keys():
        for j in constraint[1]:
            if j[0]==instance[var1] and j[1]==instance[var2]:
                return(1)
        return(0)
    else:
        return(2)
        

#print(checkConstraint(contraintes[0],{'x1':1,'x2':2}))

<h5>Heuristiques de sélection de variables pour backtrack </h5>

In [39]:
"Heuristics for variable selection in backtracking algorithm"
def strategy_selection_1(variables,instance):
    #The minimum index
     for x in variables.keys():
        if not(x in instance.keys()):
            return x
def strategy_selection_2(variables,instance):
    #The maximum index
     for x in sorted(list(variables.keys()),reverse=True):
        if not(x in instance.keys()):
            return x        
def strategy_selection_3(variables,instance):
    #Most constrained variable, min domain
    #Minimum remaining values variable first    
    keys_dict=list(variables.keys())
    values_list=list(variables.values())
    values_list_card=[len(i) for i in values_list]
    keys_var_dict_in_order=[keys_dict for _,keys_dict in sorted(zip(values_list_card,keys_dict))]
    #print(keys_var_dict_in_order)
    for x in keys_var_dict_in_order:
        if not(x in instance.keys()):
            return x
        
#pre-processing à faire avant de lancer le backtrack
def build_dict_var_number_constraints(variables,constraints):
    dict_var_number_constraints=dict()
    for var in variables.keys():
        dict_var_number_constraints[var]=0
    for var in variables.keys():
        for c in constraints:
            if var in c[0]:
                dict_var_number_constraints[var]+=1
    return dict_var_number_constraints

def strategy_selection_4(variables,instance,dict_var_number_constraints=False):
    #Most constraining variable
    #variable with most constraints on remaining variables
    #most edges in graph 
    #remaining_keys_variables=list(set(variables.keys())-set(instance.keys()))
    #build_dict_var_number_constraintsn pre-processing..
    if not dict_var_number_constraints:
        dict_var_number_constraints=build_dict_var_number_constraints(variables,constraints)
    elif isinstance(dict_var_number_constraints, dict):
        list_var_in_order=sorted(dict_var_number_constraints.items(),key=lambda t:t[1],reverse=True)
        list_var_in_order=[i[0] for i in list_var_in_order ]
        for x in list_var_in_order:
            if not(x in instance.keys()):
                return x
    else:
        return False
                
def strategy_selection_5(variables,instance,dict_var_number_constraints=False):
    #least constraining value heuristic
    if not dict_var_number_constraints:
        dict_var_number_constraints=build_dict_var_number_constraints(variables,constraints)
    elif isinstance(dict_var_number_constraints, dict):
        list_var_in_order=sorted(dict_var_number_constraints.items(),key=lambda t:t[1],reverse=False)
        list_var_in_order=[i[0] for i in list_var_in_order ]
        for x in list_var_in_order:
            if not(x in instance.keys()):
                return x
    else:
        return False

def strategy_selection_6(variables,instance):
    #Choose the variable with the minimum ratio domain size/number of constraints
    remaining_keys_variables=list(set(variables.keys())-set(instance.keys()))
    return random.choice(remaining_keys_variables)

#Pre-processing before backtrack if you use strategy_selection_7
def build_dict_var_ratio_domain_constraints(variables,constraints):
    #build the dict with the ratios domain size/number of constraints
    dict_ratios=dict()
    dict_var_number_constraints=build_dict_var_number_constraints(variables,constraints)
    #remaining_keys_variables=list(set(variables.keys())-set(instance.keys()))
    for var in variables.keys():
        if dict_var_number_constraints[var]!=0:
            dict_ratios[var]=len(variables[var])/dict_var_number_constraints[var]
        else:
            #100 is arbitrary. A big number
            dict_ratios[var]=len(variables[var])+100
    return dict_ratios

def strategy_selection_7(variables,instance,dict_ratios=False):
    #Choose a random variable with the minimul ratio 
    #(Nombre de valeurs restantes)/(Nombre de contraintes le concernant)
    if not dict_ratios:
        dict_ratios=build_dict_var_ratio_domain_constraints(variables,constraints)
    elif isinstance(dict_ratios, dict):
        list_var_in_order=sorted(dict_ratios.items(),key=lambda t:t[1],reverse=False)
        list_var_in_order=[i[0] for i in list_var_in_order ]
        for x in list_var_in_order:
            if not(x in instance.keys()):
                return x
    else:
        return False

#Impact-based Search: no time!    

def selectVarAj(variables,instance,strategy_number,dict_var_number_constraints=False,dict_ratios=False):
    if strategy_number==1:
        return strategy_selection_1(variables,instance)
    elif strategy_number==2:
        return strategy_selection_2(variables,instance)
    elif strategy_number==3:
        return strategy_selection_3(variables,instance)
    elif strategy_number==4:
        #if not dict_var_number_constraints:
        #    return strategy_selection_1(variables,instance)
        #else:
        return strategy_selection_4(variables,instance,dict_var_number_constraints)
    elif strategy_number==5:
        #if not dict_var_number_constraints:
        #    return strategy_selection_1(variables,instance)
        #else:
        return strategy_selection_5(variables,instance,dict_var_number_constraints)
    elif strategy_number==6:
        return strategy_selection_6(variables,instance)
    elif strategy_number==7:
        #if not dict_var_number_constraints:
        #    return strategy_selection_1(variables,instance)
        #:
        return strategy_selection_7(variables,instance,dict_ratios)

In [83]:
solutions=[]
#counter_calls=0
def backTrack(variables,instance,contraintes,
              strategy_number,
              dict_var_number_constraints=False,
              dict_ratios=False,
              exhaustif=False,
              forward=False):
    #global counter_calls
    #counter_calls+=1
    dict_var_number_constraints_copy=dict_var_number_constraints
    dict_ratios_copy=dict_ratios
    forward_copy=forward
    backTrack.counter_calls+=1
    complet=True
    for c in contraintes:
        ch=checkConstraint(c,instance)
        if ch==0:
            #print(ch,False,"instance fausse !!!")
            return(False)
        if ch==2:
            #print("instance incomplete")
            complet=False
    if complet==True:
        #print("instance complete")
        #print("resultat:",instance)
        solutions.append(instance)
        return(True)
    #On choisit une variable non instancée
    varAj=selectVarAj(variables,instance,strategy_number,
                      dict_var_number_constraints=dict_var_number_constraints_copy,
                      dict_ratios=dict_ratios_copy)
    #reordonner eventuellement varAj avant
    #Pour chaque valeur y de varAj dans son domaine
    for y in variables[varAj]:
        #print(varAj,"varAj ")
        #print(y,"y")
        instance2=instance.copy()
        variables2=variables.copy()
        instance2[varAj]=y
        if forward:
            variables2=forwardChecking(instance2,variables2,contraintes,varAj)
        #print("instance actuelle",instance2)
        val=backTrack(variables2,instance2,contraintes,strategy_number,
                      dict_var_number_constraints=dict_var_number_constraints_copy,
                      dict_ratios=dict_ratios_copy,
                      exhaustif=False,
                      forward=forward_copy)
        if val and exhaustif==False:
            return(True)
    return (False)

In [66]:
""""instance=dict()
variables={}
variables["x1"]=[1,2]
variables["x2"]=[1,2]
contraintes=[[("x1","x2"),[(1,2),(2,2)]]]
print(backTrack(instance,contraintes,exhaustif=False))

print(instance,"instance1")

print(variables)
#print(variables["x1"])"""
#Test
#remarque faut avoir executer le code lié à la fct reine avant pr que ca tourne.
solutions=[]
#counter_calls=0
backTrack.counter_calls=0
variables,contraintes=reines(15)
instance=dict()
instance={}
#print(backTrack(instance,contraintes,exhaustif=False))
#solutions=[]
dict_ratios_vars=build_dict_var_ratio_domain_constraints(variables,contraintes)
dict_var_number_constraints_copy=build_dict_var_ratio_domain_constraints(variables,contraintes)
#strat_number=7
#backTrack(instance,contraintes,strategy_number=strat_number,
#          dict_var_number_constraints=dict_var_number_constraints_copy,
#          dict_ratios=dict_ratios_vars,
#          exhaustif=False)
#print('number of calls',backTrack.counter_calls)

In [89]:
solutions=[]
variables,contraintes=reines(13)
instance=dict()
instance={}
dict_ratios_vars=build_dict_var_ratio_domain_constraints(variables,contraintes)
dict_var_number_constraints_copy=build_dict_var_ratio_domain_constraints(variables,contraintes)
for i in range(1,8):
    print("*--*-*-*-*-")
    print('Strategie de branchement numéro',i)
    solutions=[]
    backTrack.counter_calls=0
    variables,contraintes=reines(13)
    ac_3_bis(variables,contraintes)
    if i in [4,5,7]:
        dict_ratios_vars=build_dict_var_ratio_domain_constraints(variables,contraintes)
        dict_var_number_constraints_copy=build_dict_var_ratio_domain_constraints(variables,contraintes)
    instance=dict()
    instance={}
    strat_number=i
    t0=time.time()
    backTrack(variables,instance,contraintes,strategy_number=strat_number,
          dict_var_number_constraints=dict_var_number_constraints_copy,
          dict_ratios=dict_ratios_vars,
          exhaustif=False,
          forward=False)
    t1=time.time()
    print('number of calls',backTrack.counter_calls)
    print('duration',t1-t0)

*--*-*-*-*-
Strategie de branchement numéro 1
number of calls 1366
duration 0.12408304214477539
*--*-*-*-*-
Strategie de branchement numéro 2
number of calls 1366
duration 0.05265927314758301
*--*-*-*-*-
Strategie de branchement numéro 3
number of calls 1366
duration 0.08663201332092285
*--*-*-*-*-
Strategie de branchement numéro 4
number of calls 1366
duration 0.08688187599182129
*--*-*-*-*-
Strategie de branchement numéro 5
number of calls 1366
duration 0.07848286628723145
*--*-*-*-*-
Strategie de branchement numéro 6
number of calls 807
duration 0.09100484848022461
*--*-*-*-*-
Strategie de branchement numéro 7
number of calls 1366
duration 0.08502388000488281


## AC-3

In [45]:
#Fonction séparée pour la selection du couple dans l'algo AC_3.
#Peut etre utile si existence d heuristiques interessantes
def selectCouple(aTester):
    #Different strategies can be chosen for the selection of the couple
    return aTester.pop()

In [13]:
a=[3,4,5]
selectCouple(a)
print(a)

[3, 4]


In [14]:
""""Fonction utilisee dans l'algo AC-3
Remarque: Ici, constraints n'est pas UNE contrainte mais l'ENSEMBLE des contraintes"""
def checkConstraint_ac3(constraints,instance):
    var1=list(instance.keys())[0]
    var2=list(instance.keys())[1]
    for c in constraints:
        if var1 in c[0] and var2 in c[0]:
            #si les nombres n'apparaissent meme pas dans les possibilites, on passe
            # a l iteration suivante
            """for i in range(len(c[1])):
                #check
                print("yes")
                print(var1,var2)
                print(c[1][i])
                print("-----")
                if instance[var1]==c[1][i][0] and instance[var2]==c[1][i][1]:
                    return True
                """
            if (instance[var1],instance[var2]) in c[1]:
                return True
    return False

Remarque: nouveau code ac_3, semble bien fonctionner. Testé sur l exemple du cours sur la coloration de graphe.

In [15]:
"""
Algorithme AC-3
quand on supprime une valeur a la variable x,
on ne reexamine que les variables liees a x par une contrainte"""
def ac_3(variables,constraints):
    aTester=[]
    """Initialisation aTester"""
    for i in range(len(constraints)):
       # print('constraints[i]:',constraints[i])
        #on ajoute le couple (x,y)
        aTester.append(constraints[i][0])
        #print('Atester:',aTester)
    while len(aTester)!=0:
        """"Selection du couple """
        couple=selectCouple(aTester)
        list_values_without_support=[]
        #print('-*-*-*-*-*-*-')
        #print('new couple')
        #print('list no support',list_values_without_support)
        #pr  chaque valeur pour la variable x
        #print('couple:',couple)
        """"Parcourts des valeurs de x """
        for value_x in variables[couple[0]]:
            #print('nouvelle valeur de x',couple[0],value_x)
            support=False
            """"Parcourt des valeurs de y """
            for value_y in variables[couple[1]]:
                """"On verifie qu'il existe une contrainte Cx,y validee"""
                if checkConstraint_ac3(contraintes,{couple[0]:value_x,couple[1]:value_y}):
                    support=True
                    break;
            #print('support',support,couple[0],value_x,couple[1])
            """"On cree la liste des valeurs de x sans support"""
            if not support:
                #print('x', couple[0])
                list_values_without_support.append(value_x)
                #print('list no support',list_values_without_support)

        if len(list_values_without_support)!=0:
            #print('===')
            #print('reduce domaine of', couple[0], list_values_without_support)
            """"On met a jour le domaine de x """
            variables[couple[0]]=list(set(variables[couple[0]])-set(list_values_without_support))
            #print('variables: ',variables)
            """"On met a jour aTester"""
            for i in range(len(constraints)):
                if constraints[i][0][0]!=couple[1] and constraints[i][0][1]==couple[0]:
                    #print('couple étudié ', couple)
                    #print('ajout dans aTester',constraints[i][0])
                    aTester.append([constraints[i][0][0],constraints[i][0][1]])
                    #print('======')
            #print('======')
        #print('-*-*-*-*-*-*-')
       
                    
                                
    return True
            

In [16]:
variables=dict()
variables={}
variables["x1"]=[1,2,3]
variables["x2"]=[1,2]
variables["x3"]=[1]
contraintes=[[("x1","x2"),[(1,2),(2,1),(3,1),(3,2)]],[("x2","x1"),[(1,2),(1,3),(2,1),(2,3)]],
             [("x2","x3"),[(2,1)]],[("x3","x2"),[(1,2)]],
            [("x1","x3"),[(2,1),(3,1)]],[("x3","x1"),[(1,2),(1,3)]]
            ]
t1 = time.time()
ac_3(variables,contraintes)
t2=time.time()
duree=t2-t1

In [17]:
variables

{'x1': [3], 'x2': [2], 'x3': [1]}

In [18]:
duree

0.0001342296600341797

In [42]:
""""Fonction utilisee dans l'algo AC_3_Bis
Remarque: Ici, constraints n'est pas UNE contrainte mais l'ENSEMBLE des contraintes"""
def checkConstraint_ac3_bis(variables, constraints,var1,value_var1,var2):
    for c in constraints:
        if var1 in c[0] and var2 in c[0]:
            for value_var2 in variables[var2]:
                if (value_var1,value_var2) in c[1]:
                    return True
    return False

In [43]:
"""
Algorithme AC-3
quand on supprime une valeur a la variable x,
on ne reexamine que les variables liees a x par une contrainte"""
"""" Difference avec ac_3 ? au niveau de la fonction checkConstraint_ac3, ordre for c in constraints
et for value_y in domaines de y"""
""""Performances ac_3_bis peut etre legerement meilleur que ac_3"""
def ac_3_bis(variables,constraints):
    aTester=[]
    """Initialisation aTester"""
    for i in range(len(constraints)):
       # print('constraints[i]:',constraints[i])
        #on ajoute le couple (x,y)
        aTester.append(constraints[i][0])
    while len(aTester)!=0:
        """"Selection du couple """
        couple=selectCouple(aTester)
        list_values_without_support=[]
        #print('-*-*-*-*-*-*-')
        #print('new couple')
        #print('list no support',list_values_without_support)
        #pr  chaque valeur pour la variable x
        #print('couple:',couple)
        """"Parcourts des valeurs de x """
        for value_x in variables[couple[0]]:
            #print('nouvelle valeur de x',couple[0],value_x)
            support=False
            """"On verifie qu'il existe une contrainte Cx,y validee"""
            if checkConstraint_ac3_bis(variables,constraints,couple[0],value_x,couple[1]):
                support=True
                break;
            #print('support',support,couple[0],value_x,couple[1])
            """"On cree la liste des valeurs de x sans support"""
            if not support:
                #print('x', couple[0])
                list_values_without_support.append(value_x)
                #print('list no support',list_values_without_support)

        if len(list_values_without_support)!=0:
            #print('===')
            #print('reduce domaine of', couple[0], list_values_without_support)
            """"On met a jour le domaine de x """
            variables[couple[0]]=list(set(variables[couple[0]])-set(list_values_without_support))
            #print('variables: ',variables)
            """"On met a jour aTester"""
            for i in range(len(constraints)):
                if constraints[i][0][0]!=couple[1] and constraints[i][0][1]==couple[0]:
                    #print('couple étudié ', couple)
                    #print('ajout dans aTester',constraints[i][0])
                    aTester.append([constraints[i][0][0],constraints[i][0][1]])
                    #print('======')
            #print('======')
        #print('-*-*-*-*-*-*-')
       
                    
                                
    return True
            

## Forward checking

In [64]:
"""Forward Checking
idéé: anticiper certaines prises de decisions dans le backtrack.
Dès qu’une variable x est instanciee à la valeur a, 
on filtre toutes les valeurs incompatibles avec〈x,a〉mais on ne propage pas plus.
Remarque: Ici, constraints n'est pas UNE contrainte mais l'ENSEMBLE des contraintes
var1 est la variable que l'on analyse.
"""
def forwardChecking_old(instance_partielle,variables,constraints,var1):
    for var2 in variables.keys():
        list_no_support=[]
        if var2 not in instance_partielle.keys():
            exist_constraint=False
            for c in constraints:
                if (var1,var2)==c[0]:
                    exist_constraint=True
                    break
            if exist_constraint:
                for value_var2 in variables[var2]:
                    #for loop supplementaire dans le cas où les contraintes sur x1,x2 ne sont 
                    #pas regroupees dans un seul bloc de constraints. A modifier si c est le cas
                    #finalement
                    for c in constraints:
                        if (var1,var2)==c[0] and not (instance_partielle[var1],value_var2) in c[1]:
                            list_no_support.append(value_var2)
        if len(list_no_support)!=0:
            variables[var2]=list(set(variables[var2])-set(list_no_support))
    return True


In [65]:
"""Forward Checking
idéé: anticiper certaines prises de decisions dans le backtrack.
Dès qu’une variable x est instanciee à la valeur a, 
on filtre toutes les valeurs incompatibles avec〈x,a〉mais on ne propage pas plus.
Remarque: Ici, constraints n'est pas UNE contrainte mais l'ENSEMBLE des contraintes
var1 est la variable que l'on analyse.
"""
def forwardChecking(instance_partielle,variables,constraints,var1):
    variablesC=variables.copy()
    print(constraints)
    for var2 in variablesC.keys():
        list_no_support=[]
        if var2 not in instance_partielle.keys():
            exist_constraint=False
            for c in constraints:
                if (var1,var2)==c[0]:
                    exist_constraint=True
                    break
            if exist_constraint:
                for value_var2 in variablesC[var2]:
                    #for loop supplementaire dans le cas où les contraintes sur x1,x2 ne sont 
                    #pas regroupees dans un seul bloc de constraints. A modifier si c est le cas
                    #finalement
                    for c in constraints:
                        if (var1,var2)==c[0] and not (instance_partielle[var1],value_var2) in c[1]:
                            list_no_support.append(value_var2)
        if len(list_no_support)!=0:
            variablesC[var2]=list(set(variablesC[var2])-set(list_no_support))
    return variablesC

In [28]:
"""TEST"""
variables=dict()
variables={}
variables["x1"]=[1,2]
variables["x2"]=[1,2]
variables["x3"]=[1,3]
variables["x4"]=[1,2]
contraintes=[[("x1","x2"),[(1,2),(2,1),(3,1),(3,2)]],[("x2","x1"),[(1,2),(1,3),(2,1),(2,3)]],
             [("x2","x3"),[(2,1)]],[("x3","x2"),[(1,2)]],
            [("x1","x3"),[(2,1),(3,1)]],[("x3","x1"),[(1,2),(1,3)]],
            [("x1","x4"),[(1,2),(2,1),(3,1),(3,2)]],[("x4","x1"),[(1,2),(1,3),(2,1),(2,3)]],
            [("x3","x4"),[(1,2)]],[("x4","x3"),[(2,1)]],
            ]



In [29]:
instance_partielle={"x1":3}

In [30]:
forwardChecking_old(instance_partielle,variables,contraintes,"x1")

True

In [31]:
variables

{'x1': [1, 2], 'x2': [1, 2], 'x3': [1], 'x4': [1, 2]}

## AC-4

In [93]:
def initAC4(variables,contraintes):
    Q=set()
    S=dict()
    S={}
    Count=dict()
    Count={}
    for c in contraintes:
        x=c[0][0]
        y=c[0][1]
        for a in variables[x]:
            total=0
            Count[(x,y,a)]=0
            for b in variables[y]:
                if (a,b) in c[1]:
                    total=total+1
                    if (y,b) in S.keys():
                        S[(y,b)].append((x,a))
                    else:
                        S[(y,b)]=[(x,a)]
                Count[(x,y,a)]=total
            if Count[(x,y,a)]==0:
                variables[x].remove(a)
                A=set([(x,a)])
                Q=Q|A
    return(Q,S,Count)
        

In [97]:
def AC4(variables,contraintes):
    Q,S,Count=initAC4(variables,contraintes)
    #print("Q:",Q)
    #print("S:",S)
    #print("count:", Count)
    while len(Q)>0:
        yb=Q.pop()
        y,b=yb[0],yb[1]
        if yb in S.keys():
            for xa in S[yb]:
                x,a=xa[0],xa[1]
                print(x,y,a)
                Count[(x,y,a)]=Count[(x,y,a)]-1
                if Count[(x,y,a)]==0 and a in variables[x]:
                    variables[x].remove(a)
                    A=set([(x,a)])
                    Q=Q|A
    return(variables)

In [95]:
"""TEST"""
variables=dict()
variables={}
variables["x1"]=[1,2]
variables["x2"]=[1,2]
variables["x3"]=[1,3]
variables["x4"]=[1,2]
contraintes=[[("x1","x2"),[(1,2),(2,1),(3,1),(3,2)]],[("x2","x1"),[(1,2),(1,3),(2,1),(2,3)]],
             [("x2","x3"),[(2,1),(2,3)]],[("x3","x2"),[(1,2),(3,2)]],
            [("x1","x3"),[(2,1),(3,1),(1,3)]],[("x3","x1"),[(1,2),(1,3),(3,1)]],
            [("x1","x4"),[(1,2),(2,1),(3,1),(3,2)]],[("x4","x1"),[(1,2),(1,3),(2,1),(2,3)]],
            [("x3","x4"),[(1,2),(3,2)]],[("x4","x3"),[(2,1),(2,3)]],
            ]
#backTrack(instance,contraintes)


In [4]:
print(variables)

AC4(variables,contraintes)

{'x1': [1, 2], 'x2': [1, 2], 'x3': [1, 3], 'x4': [1, 2]}
Q: {('x4', 1), ('x2', 1)}
S: {('x2', 2): [('x1', 1), ('x3', 1), ('x3', 3)], ('x2', 1): [('x1', 2)], ('x1', 2): [('x2', 1), ('x3', 1), ('x4', 1)], ('x1', 1): [('x2', 2), ('x3', 3), ('x4', 2)], ('x3', 3): [('x1', 1)], ('x3', 1): [('x1', 2)], ('x4', 2): [('x1', 1), ('x3', 1), ('x3', 3)], ('x4', 1): [('x1', 2)]}
count: {('x1', 'x2', 1): 1, ('x1', 'x2', 2): 1, ('x2', 'x1', 1): 1, ('x2', 'x1', 2): 1, ('x2', 'x3', 1): 0, ('x3', 'x2', 1): 1, ('x3', 'x2', 3): 1, ('x1', 'x3', 1): 1, ('x1', 'x3', 2): 1, ('x3', 'x1', 1): 1, ('x3', 'x1', 3): 1, ('x1', 'x4', 1): 1, ('x1', 'x4', 2): 1, ('x4', 'x1', 1): 1, ('x4', 'x1', 2): 1, ('x3', 'x4', 1): 1, ('x3', 'x4', 3): 1, ('x4', 'x3', 1): 0}
x1 x4 2
x2 x1 1
x3 x1 1
x4 x1 1
x1 x3 2
x1 x2 2


{'x1': [1], 'x2': [2], 'x3': [3], 'x4': [2]}

## Reines

In [49]:
def reines(n):
    variables=dict()
    variables={}
    contraintes=[]
    variables[1]=[j for j in range(1,(n+1)//2+1)]
    for i in range(2,n+1):
        variables[i]=[j for j in range(1,n+1)]
    for i in range(1,n+1):
        for j in range(i+1,n+1):
            c=[]
            for a in range(1,n+1):
                for b in range(1,n+1):
                    if a!=b and b-a!=j-i and a-b!=j-i:
                        c.append((a,b))
            contraintes.append([(i,j),c])
    return(variables,contraintes)

print(reines(5))
            

({1: [1, 2, 3], 2: [1, 2, 3, 4, 5], 3: [1, 2, 3, 4, 5], 4: [1, 2, 3, 4, 5], 5: [1, 2, 3, 4, 5]}, [[(1, 2), [(1, 3), (1, 4), (1, 5), (2, 4), (2, 5), (3, 1), (3, 5), (4, 1), (4, 2), (5, 1), (5, 2), (5, 3)]], [(1, 3), [(1, 2), (1, 4), (1, 5), (2, 1), (2, 3), (2, 5), (3, 2), (3, 4), (4, 1), (4, 3), (4, 5), (5, 1), (5, 2), (5, 4)]], [(1, 4), [(1, 2), (1, 3), (1, 5), (2, 1), (2, 3), (2, 4), (3, 1), (3, 2), (3, 4), (3, 5), (4, 2), (4, 3), (4, 5), (5, 1), (5, 3), (5, 4)]], [(1, 5), [(1, 2), (1, 3), (1, 4), (2, 1), (2, 3), (2, 4), (2, 5), (3, 1), (3, 2), (3, 4), (3, 5), (4, 1), (4, 2), (4, 3), (4, 5), (5, 2), (5, 3), (5, 4)]], [(2, 3), [(1, 3), (1, 4), (1, 5), (2, 4), (2, 5), (3, 1), (3, 5), (4, 1), (4, 2), (5, 1), (5, 2), (5, 3)]], [(2, 4), [(1, 2), (1, 4), (1, 5), (2, 1), (2, 3), (2, 5), (3, 2), (3, 4), (4, 1), (4, 3), (4, 5), (5, 1), (5, 2), (5, 4)]], [(2, 5), [(1, 2), (1, 3), (1, 5), (2, 1), (2, 3), (2, 4), (3, 1), (3, 2), (3, 4), (3, 5), (4, 2), (4, 3), (4, 5), (5, 1), (5, 3), (5, 4)]], [(

In [52]:
variables,contraintes=reines(5)
instance=dict()
instance={}
print(backTrack(instance,contraintes,strategy_number=1))

instance complete
resultat: {1: 1, 2: 3, 3: 5, 4: 2, 5: 4}
True


In [53]:
variables,contraintes=reines(5)
instance=dict()
instance={}

In [54]:
AC4(variables,contraintes)

Q: set()
S: {(2, 3): [(1, 1)], (2, 4): [(1, 1), (1, 2)], (2, 5): [(1, 1), (1, 2), (1, 3)], (2, 1): [(1, 3)], (3, 2): [(1, 1), (1, 3), (2, 4), (2, 5)], (3, 4): [(1, 1), (1, 3), (2, 1), (2, 2)], (3, 5): [(1, 1), (1, 2), (2, 1), (2, 2), (2, 3)], (3, 1): [(1, 2), (2, 3), (2, 4), (2, 5)], (3, 3): [(1, 2), (2, 1), (2, 5)], (4, 2): [(1, 1), (1, 3), (2, 1), (2, 3), (2, 5), (3, 4), (3, 5)], (4, 3): [(1, 1), (1, 2), (2, 2), (2, 4), (3, 1), (3, 5)], (4, 5): [(1, 1), (1, 3), (2, 1), (2, 2), (2, 4), (3, 1), (3, 2), (3, 3)], (4, 1): [(1, 2), (1, 3), (2, 2), (2, 4), (2, 5), (3, 3), (3, 4), (3, 5)], (4, 4): [(1, 2), (1, 3), (2, 1), (2, 3), (2, 5), (3, 1), (3, 2)], (5, 2): [(1, 1), (1, 3), (2, 1), (2, 3), (2, 4), (3, 1), (3, 3), (3, 5), (4, 4), (4, 5)], (5, 3): [(1, 1), (1, 2), (2, 1), (2, 2), (2, 4), (2, 5), (3, 2), (3, 4), (4, 1), (4, 5)], (5, 4): [(1, 1), (1, 2), (1, 3), (2, 2), (2, 3), (2, 5), (3, 1), (3, 3), (3, 5), (4, 1), (4, 2)], (5, 1): [(1, 2), (1, 3), (2, 2), (2, 3), (2, 5), (3, 2), (3, 4), 

{1: [1, 2, 3],
 2: [1, 2, 3, 4, 5],
 3: [1, 2, 3, 4, 5],
 4: [1, 2, 3, 4, 5],
 5: [1, 2, 3, 4, 5]}

In [56]:
variables,contraintes=reines(10)
instance=dict()
instance={}

variables=AC4(variables,contraintes)
print(variables)
print(backTrack(instance,contraintes,strategy_number=1,exhaustif=False))

Q: set()
S: {(2, 3): [(1, 1), (1, 5)], (2, 4): [(1, 1), (1, 2)], (2, 5): [(1, 1), (1, 2), (1, 3)], (2, 6): [(1, 1), (1, 2), (1, 3), (1, 4)], (2, 7): [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)], (2, 8): [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)], (2, 9): [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)], (2, 10): [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)], (2, 1): [(1, 3), (1, 4), (1, 5)], (2, 2): [(1, 4), (1, 5)], (3, 2): [(1, 1), (1, 3), (1, 5), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10)], (3, 4): [(1, 1), (1, 3), (1, 5), (2, 1), (2, 2), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10)], (3, 5): [(1, 1), (1, 2), (1, 4), (2, 1), (2, 2), (2, 3), (2, 7), (2, 8), (2, 9), (2, 10)], (3, 6): [(1, 1), (1, 2), (1, 3), (1, 5), (2, 1), (2, 2), (2, 3), (2, 4), (2, 8), (2, 9), (2, 10)], (3, 7): [(1, 1), (1, 2), (1, 3), (1, 4), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 9), (2, 10)], (3, 8): [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 10)], (3, 9): [(1, 1), (1,

## Coloration de graphes

In [101]:
def colorGraph(nSommets,aretes,nCouleurs):
    variables=dict()
    variables={}
    contraintes=[]
    combinaisons=[]
    combinaisonsDiff=[]
    for c1 in range(nCouleurs):
        for c2 in range(nCouleurs):
            combinaisons.append((c1,c2))
            if c1!=c2:
                combinaisonsDiff.append((c1,c2))
    variables[1]=[0]
    for s in range(2,nSommets+1):
        variables[s]=[k for k in range(nCouleurs)]
    for i in range(1,nSommets+1):
        for j in range(1,nSommets+1):
            if i!=j and ((i,j) in aretes or (j,i) in aretes):
                contraintes.append([(i,j),combinaisonsDiff.copy()])
            elif i!=j:
                contraintes.append([(i,j),combinaisons.copy()])
    return(variables,contraintes)

In [99]:
#print(colorGraph(6,[(1,2),(2,3),(1,3),(4,2),(4,5)],3))

In [103]:
variables,contraintes=colorGraph(6,[(0,1),(0,3),(1,2),(2,3),(1,3),(4,2),(4,5)],3)
instance=dict()
instance={}

#print("contraintes:", contraintes,"\n")

variables=AC4(variables,contraintes)
#print(variables)
print(backTrack(variables,instance,contraintes,strategy_number=1))

2 3 1
2 3 2
True


In [93]:
import os

def readGraph(nomFichier):
    fichier=open(nomFichier,"r")
    l=fichier.readline()
    tab=l.split()
    nSommets=int(tab[2])
    aretes=[]
    for l in fichier.readlines():
        tab=l.split(' ')
        d=(int(tab[1]),int(tab[2][0]))
        if not(d in aretes):
            aretes.append(d)
    return(nSommets,aretes)

nSommetsG,aretesG=readGraph("myciel3.col")

In [100]:
nSommets,aretes=nSommetsG,aretesG
print(nSommets)

variables,contraintes=colorGraph(nSommets,aretes,3)

variables=AC4(variables,contraintes)

print(backTrack(instance,contraintes,exhaustif=False))

11
Q: set()
S: {(1, 0): [(0, 0), (2, 1), (2, 2), (3, 1), (3, 2), (4, 1), (4, 2), (5, 0), (5, 1), (5, 2), (6, 1), (6, 2), (7, 1), (7, 2), (8, 1), (8, 2), (9, 1), (9, 2), (10, 1), (10, 2)], (1, 1): [(0, 0), (2, 0), (2, 2), (3, 0), (3, 2), (4, 0), (4, 2), (5, 0), (5, 1), (5, 2), (6, 0), (6, 2), (7, 0), (7, 2), (8, 0), (8, 2), (9, 0), (9, 2), (10, 0), (10, 2)], (1, 2): [(0, 0), (2, 0), (2, 1), (3, 0), (3, 1), (4, 0), (4, 1), (5, 0), (5, 1), (5, 2), (6, 0), (6, 1), (7, 0), (7, 1), (8, 0), (8, 1), (9, 0), (9, 1), (10, 0), (10, 1)], (2, 0): [(0, 0), (1, 1), (1, 2), (3, 1), (3, 2), (4, 0), (4, 1), (4, 2), (5, 0), (5, 1), (5, 2), (6, 1), (6, 2), (7, 0), (7, 1), (7, 2), (8, 1), (8, 2), (9, 0), (9, 1), (9, 2), (10, 0), (10, 1), (10, 2)], (2, 1): [(0, 0), (1, 0), (1, 2), (3, 0), (3, 2), (4, 0), (4, 1), (4, 2), (5, 0), (5, 1), (5, 2), (6, 0), (6, 2), (7, 0), (7, 1), (7, 2), (8, 0), (8, 2), (9, 0), (9, 1), (9, 2), (10, 0), (10, 1), (10, 2)], (2, 2): [(0, 0), (1, 0), (1, 1), (3, 0), (3, 1), (4, 0), (

In [ ]:
aretesG

<h1>BENCHMARKS</h1>

<h3>nReines</h3>

In [105]:
solutions=[]
variables,contraintes=reines(13)
instance=dict()
instance={}
dict_ratios_vars=build_dict_var_ratio_domain_constraints(variables,contraintes)
dict_var_number_constraints_copy=build_dict_var_ratio_domain_constraints(variables,contraintes)
for i in range(1,8):
    print("*--*-*-*-*-")
    print('Strategie de branchement numéro',i)
    solutions=[]
    backTrack.counter_calls=0
    variables,contraintes=reines(13)
    ac_3_bis(variables,contraintes)
    if i in [4,5,7]:
        dict_ratios_vars=build_dict_var_ratio_domain_constraints(variables,contraintes)
        dict_var_number_constraints_copy=build_dict_var_ratio_domain_constraints(variables,contraintes)
    instance=dict()
    instance={}
    strat_number=i
    t0=time.time()
    backTrack(variables,instance,contraintes,strategy_number=strat_number,
          dict_var_number_constraints=dict_var_number_constraints_copy,
          dict_ratios=dict_ratios_vars,
          exhaustif=False,
          forward=False)
    t1=time.time()
    print('number of calls',backTrack.counter_calls)
    print('duration',t1-t0)

*--*-*-*-*-
Strategie de branchement numéro 1
number of calls 1366
duration 0.12952637672424316
*--*-*-*-*-
Strategie de branchement numéro 2
number of calls 1366
duration 0.05509328842163086
*--*-*-*-*-
Strategie de branchement numéro 3
number of calls 1366
duration 0.0989530086517334
*--*-*-*-*-
Strategie de branchement numéro 4
number of calls 1366
duration 0.10054492950439453
*--*-*-*-*-
Strategie de branchement numéro 5
number of calls 1366
duration 0.08302974700927734
*--*-*-*-*-
Strategie de branchement numéro 6
number of calls 911
duration 0.09677505493164062
*--*-*-*-*-
Strategie de branchement numéro 7
number of calls 1366
duration 0.09147024154663086


In [106]:
""""solutions=[]
variables,contraintes=reines(6)
instance=dict()
instance={}
dict_ratios_vars=build_dict_var_ratio_domain_constraints(variables,contraintes)
dict_var_number_constraints_copy=build_dict_var_ratio_domain_constraints(variables,contraintes)"""
number_calls_recap=[0,0,0,0,0,0,0,0,0]
time_recap=[0,0,0,0,0,0,0,0,0]
for i in range(1,8):
    print("*--*-*-*-*-")
    print('Strategie de branchement numéro',i)
    for k in range(1,15):
        #print("===========")
        #print("Problème nReines avec n:",k)
        solutions=[]
        backTrack.counter_calls=0
        variables,contraintes=reines(k)
        AC4(variables,contraintes)
        #ac_3_bis(variables,contraintes)
        if i in [4,5,7]:
            dict_ratios_vars=build_dict_var_ratio_domain_constraints(variables,contraintes)
            dict_var_number_constraints_copy=build_dict_var_ratio_domain_constraints(variables,contraintes)
        instance=dict()
        instance={}
        strat_number=i
        t0=time.time()
        backTrack(variables,instance,contraintes,strategy_number=strat_number,
              dict_var_number_constraints=dict_var_number_constraints_copy,
              dict_ratios=dict_ratios_vars,
              exhaustif=False,
              forward=False)
        t1=time.time()
        #print('number of calls',backTrack.counter_calls)
        number_calls_recap[i-1]=number_calls_recap[i-1]+backTrack.counter_calls
        time_recap[i-1]=time_recap[i-1]+t1-t0
        #print('duration',t1-t0)
    print("Total call",number_calls_recap[i-1])
    print("Total time", time_recap[i-1])

*--*-*-*-*-
Strategie de branchement numéro 1
Total call 33901
Total time 3.2648656368255615
*--*-*-*-*-
Strategie de branchement numéro 2
Total call 42374
Total time 1.719986915588379
*--*-*-*-*-
Strategie de branchement numéro 3
Total call 33901
Total time 2.8421220779418945
*--*-*-*-*-
Strategie de branchement numéro 4
Total call 70898
Total time 5.884023189544678
*--*-*-*-*-
Strategie de branchement numéro 5
Total call 33901
Total time 3.817542314529419
*--*-*-*-*-
Strategie de branchement numéro 6
Total call 9661
Total time 1.0074262619018555
*--*-*-*-*-
Strategie de branchement numéro 7
Total call 33901
Total time 4.476372241973877


<h3>Colorations de Graphes </h3>

In [ ]:
def testsCol(fichier,nCol):
    
    nSommets,aretes=readGraph(fichier)
    print("probleme:", fichier)
    print("nombre de sommets:",nSommets)
    
    variables,contraintes=colorGraph(nSommets,aretes,nCol)
    
    t0=time.time()
    variables1=ac_3(variables,contraintes)
    t1=time.time()
    print("test ac3:",t1-t0)
    
    t0=time.time()
    variables2=AC4(variables,contraintes)
    t1=time.time()
    print("test ac4:",t1-t0)
    
    t0=time.time()
    backTrack.counter_calls=0
    print(backTrack(variables,instance,contraintes,strategy_number=1,exhaustif=False))
    t1=time.time()
    print("test backtrack simple:", t1-t0)
    
    t0=time.time()
    backTrack.counter_calls=0
    print(backTrack(variables,instance,contraintes,strategy_number=1,exhaustif=False,forward=True))
    t1=time.time()
    print("test backtrack forward", t1-t0)
    
    t0=time.time()
    backTrack.counter_calls=0
    print(backTrack(variables1,instance,contraintes,strategy_number=1,exhaustif=False))
    t1=time.time()
    print("test backtrack simple, apres AC3:", t1-t0)
    
    t0=time.time()
    backTrack.counter_calls=0
    print(backTrack(variables2,instance,contraintes,strategy_number=1,exhaustif=False,forward=True))
    t1=time.time()
    print("test backtrack forward, apres AC4", t1-t0)

Comparaison stratégies de branchements pour myciel3.col

In [ ]:
for i in range(1,8):
    print("*--*-*-*-*-")
    print('Strategie de branchement numéro',i)
    solutions=[]
    backTrack.counter_calls=0
    variables,contraintes=reines(13)
    nSommets,aretes=readGraph(myciel3.col)
    variables,contraintes=colorGraph(nSommets,aretes,nCol)
    ac_3_bis(variables,contraintes)
    if i in [4,5,7]:
        dict_ratios_vars=build_dict_var_ratio_domain_constraints(variables,contraintes)
        dict_var_number_constraints_copy=build_dict_var_ratio_domain_constraints(variables,contraintes)
    instance=dict()
    instance={}
    strat_number=i
    t0=time.time()
    backTrack(variables,instance,contraintes,strategy_number=strat_number,
          dict_var_number_constraints=dict_var_number_constraints_copy,
          dict_ratios=dict_ratios_vars,
          exhaustif=False,
          forward=False)
    t1=time.time()
    print('number of calls',backTrack.counter_calls)
    print('duration',t1-t0)